In [1]:
import urllib2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from urlparse import urlsplit, urljoin

In [35]:
base_url = 'http://www.cra-arc.gc.ca/formspubs/t1gnrl/llyrs-eng.html'
base_page = urllib2.urlopen(base_url).read()
selection = Selector(text=base_page)

### Get a url for each year

In [36]:
year_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/@href").extract()

In [68]:
url_split = urlsplit(url)
joined_urls = []
for year_url in year_pages:
    # Create the url and then append it into the empty array
    url_joined = 'http://' + url_split.netloc + year_url
    joined_urls.append(url_joined)

# Debugging
# joined_urls

In [69]:
provincial_links = []

def provinceUrlListAppend(url_list, flag, year_url):
    if flag == "normal" or flag == "older":
        for url in url_list:
            provincial_links.append('http://' + url_split.netloc + url)
    elif flag == "reallyOld":
        for url in url_list:
            link = 'http://' + url_split.netloc
            link += urlsplit(year_url).path.rsplit('/', 1)[0]
            link += "/" + url
            provincial_links.append(link)

def provinceUrlListProcess(url_list, year_url):
    # If it's 'normal' and Alberta is detected, go ahead
    if url_list[0] == 'Alberta' or url_list[0] == ' ':
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[1]/li/a/@href").extract()
        provinceUrlListAppend(url_list, "normal", year_url)
    else:
        # In some older links, they stuck another unordered list above
        url_list = selection.xpath("//main[@property='mainContentOfPage']/ul[2]/li/a/@href").extract()
        # Another edge case check where they started using relative urls around 1998
        if url_list[0][0] == "/":
            provinceUrlListAppend(url_list, "older", year_url)
        else:
            provinceUrlListAppend(url_list, "reallyOld", year_url)

for year_url in joined_urls:
    year_page = urllib2.urlopen(year_url).read()
    selection = Selector(text=year_page)
    # This checks for an edge case where some pages have the province urls placed in a differnt location
    provt_pages = selection.xpath("//main[@property='mainContentOfPage']/ul/li/a/text()").extract()
    print year_url
    provinceUrlListProcess(provt_pages, year_url)

# Debugging  
# provincial_links

http://www.cra-arc.gc.ca/formspubs/t1gnrl/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2013/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2012/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2011/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2010/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2009/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2008/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2007/menu-eng.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2006/menu-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2005/menu-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2004/menu-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2003/menu-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2002/menu-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2001/menu-e.html
http://www.cra-arc.gc.ca/formspubs/prioryear/t1/2000/menu-e.html
http://www.cra-arc.g

In [52]:
print joined_urls[20]
joined_urls[20].rsplit('/', 1)[0]

http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1994/menu-e.html


u'http://www.cra-arc.gc.ca/formspubs/prioryear/t1/1994'

In [56]:
urlsplit(joined_urls[20]).path.rsplit('/', 1)[0]

u'/formspubs/prioryear/t1/1994'

In [58]:
provt_pages[0][0]

u'a'